In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pprint
import re
import pandas as pd

In [13]:
p=0
url = f'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&object_type%5B0%5D=1&offer_type=flat&p={p}&region=1&room1=1'


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.5.1027 Yowser/2.5 Safari/537.36',
}

rooms_number=[]
flat_type=[]
total_area = []
flat_floor =[]
house_total_floor =[]
metro=[]
time_to_metro_minets=[]
way_to_metro=[]
price_main=[]
price_main_value=[]
price_meter_value=[]
price_meter=[]
adress=[]


In [14]:
for j in range(1,100):
    response = requests.get(url, headers=headers)
    soup = bs(response.content, 'html.parser')
    mylist=soup.find_all('div',class_=re.compile("--card--"))
    for i in range(len(mylist)): 
        try:
            house_info_i=mylist[i].find('div',{"data-name": "GeneralInfoSectionRowComponent"}).find('div',class_=re.compile("--subtitle--")).find('span').text
            rooms_number.append(int(house_info_i[:house_info_i.find('-')]))
            flat_type.append(house_info_i[house_info_i.find('.')+2:house_info_i.find(',')])
            total_area.append(float((house_info_i[house_info_i.find(',')+2:house_info_i.find('м²')-1]).replace(',', '.')))
            flat_floor.append(int(house_info_i[house_info_i.find('м², ')+4:house_info_i.find('/')]))
            house_total_floor.append(int(house_info_i[house_info_i.find('/')+1:house_info_i.find(' этаж')]))
            
            metro_i=mylist[i].find('div',{"data-name": "SpecialGeo"}).find_all('div')[-2].text
            metro.append(metro_i)
            metro_1_i=mylist[i].find('div',{"data-name": "SpecialGeo"}).find_all('div')[-1].text
            time_to_metro_minets.append(int(metro_1_i[:metro_1_i.find(' ')]))
            
            if metro_1_i[metro_1_i.rfind(' ')+1:] == 'транспорте':
                way = 'на '+metro_1_i[metro_1_i.rfind(' ')+1:]
            else:
                way = metro_1_i[metro_1_i.rfind(' ')+1:]
            
            way_to_metro.append(way)
            
            price_main_i=mylist[i].find('span',{"data-mark": "MainPrice"}).find('span').text
            price_main_value_i = price_main_i[-1] 
            price_main_value.append(price_main_value_i)
            price_main_i=int(price_main_i[:price_main_i.find('\xa0₽')].replace(' ', ''))
            price_main.append(price_main_i)
            price_main_meter_i=mylist[i].find('p',{"data-mark": "PriceInfo"}).text
            price_meter_value_i = price_main_meter_i[price_main_meter_i.find('\xa0')+1:]
            price_meter_value.append(price_meter_value_i)
            price_meter_i = int(price_main_meter_i[:price_main_meter_i.find('\xa0')].replace(' ', ''))
            price_meter.append(price_meter_i)
            res_adress = mylist[i].find_all('a',{"data-name": "GeoLabel"})
            adres = ''
            for i in res_adress:
                adres= adres+i.text+','
            adress_i =adres[:-1]
            adress.append(adress_i)
        except:
            mylist[i]=''
    p+=1
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


In [40]:
data = {
        'rooms_number':rooms_number,
        'flat_type':flat_type,
        'total_area' : total_area,
        'flat_floor' :flat_floor,
        'house_total_floor' : house_total_floor,
        'metro': metro,
        'time_to_metro_minets':time_to_metro_minets,
        'way_to_metro':way_to_metro,
        'price_main': price_main,
        'price_main_value': price_main_value,
        'price_meter': price_meter,
        'price_meter_value': price_meter_value,
        'adress': adress}  
df = pd.DataFrame(data) 

df.loc[(df["adress"] == ''), ("adress")] = 'Москва'

df

,rooms_number,flat_type,total_area,flat_floor,house_total_floor,metro,time_to_metro_minets,way_to_metro,price_main,price_main_value,price_meter,price_meter_value,adress
0,1,кв.,39.5,2,14,Добрынинская,3,пешком,18400000,₽,465823,₽/м²,"Москва,ЦАО,р-н Замоскворечье,м. Добрынинская,Л..."
1,1,апарт.,15.6,2,13,Маяковская,10,пешком,8980000,₽,575641,₽/м²,"Москва,ЦАО,р-н Пресненский,м. Маяковская,Зооло..."
2,1,апарт.,13.0,2,13,Баррикадная,8,пешком,7400000,₽,569231,₽/м²,"Москва,ЦАО,р-н Пресненский,м. Баррикадная,Зоол..."
3,1,апарт.,33.4,1,5,Щербинка,5,на транспорте,5362370,₽,160550,₽/м²,"Москва,НАО (Новомосковский),м. Щербинка,Щербин..."
4,1,кв.,35.0,1,12,Селигерская,10,пешком,9600000,₽,274286,₽/м²,"Москва,САО,р-н Бескудниковский,м. Селигерская,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1,кв.,43.0,13,24,Полежаевская,8,пешком,21900000,₽,509302,₽/м²,"Москва,САО,р-н Хорошевский,м. Полежаевская,Хор..."
1096,1,кв.,30.0,2,9,Новые Черемушки,12,пешком,12600000,₽,420000,₽/м²,"Москва,ЮЗАО,р-н Ломоносовский,м. Новые Черемуш..."
1097,1,кв.,35.0,8,9,Новогиреево,9,пешком,10500000,₽,300000,₽/м²,"Москва,ВАО,р-н Ивановское,м. Новогиреево,улица..."
1098,1,кв.,45.3,13,21,Раменки,15,пешком,26400000,₽,582781,₽/м²,"Москва,ЗАО,р-н Раменки,м. Раменки,Мосфильмовск..."


In [41]:
df.to_excel("output.xlsx",
             sheet_name='Sheet_name_1')  

In [42]:
df.to_csv("output.csv", sep='\t')